In [2]:
import heapq
import math

class State:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __eq__(self, other):
        return isinstance(other, State) and self.x == other.x and self.y == other.y
    def __hash__(self):
        return hash((self.x, self.y))
    def __repr__(self):
        return f"({self.x},{self.y})"

DIRECTIONS = [(-1, -1), (-1, 0), (-1, 1),
              (0, -1),          (0, 1),
              (1, -1),  (1, 0), (1, 1)]

def heuristic(s: State, goal: State):
    """Euclidean distance"""
    return math.sqrt((s.x - goal.x) ** 2 + (s.y - goal.y) ** 2)

def GoalTest(s: State, goal: State):
    return s == goal

def MoveGen(s: State, grid):
    """Return a list of valid neighboring states instead of a generator"""
    n = len(grid)
    neighbors = []
    for dx, dy in DIRECTIONS:
        nx, ny = s.x + dx, s.y + dy
        if 0 <= nx < n and 0 <= ny < n and grid[nx][ny] == 0:
            neighbors.append(State(nx, ny))
    return neighbors

def reconstruct_path(PARENT, goal: State):
    path = []
    cur = goal
    while cur is not None:
        path.append(cur)
        cur = PARENT.get(cur)
    return path[::-1]

# ---------------- Best First Search ----------------
def BestFirstSearch(grid):
    n = len(grid)
    start, goal = State(0, 0), State(n - 1, n - 1)
    if grid[0][0] == 1 or grid[n - 1][n - 1] == 1:
        return -1, []

    OPEN = []  # (h, tie, node, parentNode)
    counter = 0
    heapq.heappush(OPEN, (heuristic(start, goal), counter, start, None))
    CLOSED = set()
    PARENT = {}

    while OPEN:
        hN, _, N, parentNode = heapq.heappop(OPEN)
        if N in CLOSED:
            continue

        PARENT[N] = parentNode

        if GoalTest(N, goal):
            return len(reconstruct_path(PARENT, N)), reconstruct_path(PARENT, N)

        CLOSED.add(N)

        for M in MoveGen(N, grid):
            if M not in CLOSED:
                counter += 1
                heapq.heappush(OPEN, (heuristic(M, goal), counter, M, N))

    return -1, []

def PropagateImprovement(M: State, parent, g, grid):
    for C in MoveGen(M, grid):
        if parent.get(C) == M:
            new_g = g[M] + 1
            if new_g < g.get(C, float("inf")):
                g[C] = new_g
                PropagateImprovement(C, parent, g, grid)

def AStarSearch(grid):
    n = len(grid)
    start, goal = State(0, 0), State(n - 1, n - 1)
    if grid[0][0] == 1 or grid[n - 1][n - 1] == 1:
        return -1, []

    g = {start: 0}
    parent = {start: None}
    OPEN = []  # (f, tie, node)
    counter = 0
    heapq.heappush(OPEN, (g[start] + heuristic(start, goal), counter, start))
    CLOSED = set()

    while OPEN:
        fN, _, N = heapq.heappop(OPEN)
        if N in CLOSED:
            continue
        CLOSED.add(N)

        if GoalTest(N, goal):
            return len(reconstruct_path(parent, N)), reconstruct_path(parent, N)

        for M in MoveGen(N, grid):
            tentative_g = g[N] + 1
            if tentative_g < g.get(M, float("inf")):
                parent[M] = N
                g[M] = tentative_g
                if M in CLOSED:
                    PropagateImprovement(M, parent, g, grid)
                else:
                    counter += 1
                    heapq.heappush(OPEN, (g[M] + heuristic(M, goal), counter, M))

    return -1, []


if __name__ == "__main__":
    grids = [
        [[0, 1],
         [1, 0]],

        [[0, 0, 0],
         [1, 1, 0],
         [1, 1, 0]],

        [[1, 0, 0],
         [1, 1, 0],
         [1, 1, 0]],
    ]

    for i, grid in enumerate(grids, 1):
        print(f"\nExample {i}:")
        bfs_len, bfs_path = BestFirstSearch(grid)
        print(f"Best First Search  → Path length: {bfs_len}, Path: {bfs_path}")
        astar_len, astar_path = AStarSearch(grid)
        print(f"A* Search          → Path length: {astar_len}, Path: {astar_path}")
"""
---------------------- Comparison ----------------------

Best First Search (Greedy Search):
- Uses only the heuristic value h(n) to decide which node to expand.
- Very fast in practice, since it always moves closer to the goal.
- However, it does not guarantee the shortest path (may take a longer route).

A* Search:
- Uses both g(n) (cost so far) and h(n) (heuristic).
- Guarantees the shortest path if the heuristic is admissible (never overestimates).
- Slightly slower than Best First Search, but much more reliable.

Conclusion:
Best First Search is faster but may return suboptimal paths.
A* is optimal and complete — it always finds the shortest path if one exists.
---------------------------------------------------------
"""


Example 1:
Best First Search  → Path length: 2, Path: [(0,0), (1,1)]
A* Search          → Path length: 2, Path: [(0,0), (1,1)]

Example 2:
Best First Search  → Path length: 4, Path: [(0,0), (0,1), (1,2), (2,2)]
A* Search          → Path length: 4, Path: [(0,0), (0,1), (1,2), (2,2)]

Example 3:
Best First Search  → Path length: -1, Path: []
A* Search          → Path length: -1, Path: []


'\n---------------------- Comparison ----------------------\n\nBest First Search (Greedy Search):\n- Uses only the heuristic value h(n) to decide which node to expand.\n- Very fast in practice, since it always moves closer to the goal.\n- However, it does not guarantee the shortest path (may take a longer route).\n\nA* Search:\n- Uses both g(n) (cost so far) and h(n) (heuristic).\n- Guarantees the shortest path if the heuristic is admissible (never overestimates).\n- Slightly slower than Best First Search, but much more reliable.\n\nConclusion:\nBest First Search is faster but may return suboptimal paths.\nA* is optimal and complete — it always finds the shortest path if one exists.\n---------------------------------------------------------\n'